# Retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Retrievers accept a string query as input and return a list of Document's as output.

In [1]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

In [2]:
loaders = [
    TextLoader("data/langchain.md"),
    TextLoader("data/langchain2.md"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [3]:
# Define a text splitter that will be used to create child documents from larger parent documents.
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

# Initialize a vector store named "full_documents" which will index the child chunks of the documents.
# The OllamaEmbeddings model "snowflake-arctic-embed:33m" is used to generate embeddings for these chunks.
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# Set up an in-memory storage layer that will store the parent documents.
store = InMemoryStore()

# Create a retriever that uses the previously defined vector store, document store, and child splitter.
# This retriever will be able to fetch relevant parent documents based on queries and split them into child chunks as needed.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

In [4]:
retriever.add_documents(docs, ids=None)
list(store.yield_keys())

['caa9cc1b-79b0-443d-8fe9-56625ecaf0ab',
 'fb954316-151c-44e9-bdf7-aec6ed274bc5']

In [5]:
sub_docs = vectorstore.similarity_search("LangChian")


In [6]:
retrieved_docs = retriever.invoke("LangChian")
len(retrieved_docs[0].page_content)



21344

## Retrieving larger chunks

In [7]:
# This text splitter is used to create the parent documents
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# This text splitter is used to create the child documents
# It should create documents smaller than the parent
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OllamaEmbeddings(model="snowflake-arctic-embed:33m")
)
# The storage layer for the parent documents
store = InMemoryStore()

In [8]:
# Create a retriever that uses the previously defined vector store, document store, child splitter, and parent splitter.
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)


In [9]:
retriever.add_documents(docs)
len(list(store.yield_keys()))


22

In [10]:
sub_docs = vectorstore.similarity_search("LangChian")

print(sub_docs[0].page_content)

and more. Please see the [Runnable Interface](https://python.langchain.com/docs/concepts/runnables/) for more details.


In [11]:
retrieved_docs = retriever.invoke("LangChian")

len(retrieved_docs[0].page_content)
print(retrieved_docs[0].page_content)

LangChain chat models implement the [BaseChatModel](https://python.langchain.com/api_reference/core/language_models/langchain_core.language_models.chat_models.BaseChatModel.html) interface. Because `BaseChatModel` also implements the [Runnable Interface](https://python.langchain.com/docs/concepts/runnables/), chat models support a [standard streaming interface](https://python.langchain.com/docs/concepts/streaming/), [async programming](https://python.langchain.com/docs/concepts/async/), optimized [batching](https://python.langchain.com/docs/concepts/runnables/#optimized-parallel-execution-batch), and more. Please see the [Runnable Interface](https://python.langchain.com/docs/concepts/runnables/) for more details.

Many of the key methods of chat models operate on [messages](https://python.langchain.com/docs/concepts/messages/) as input and return messages as output.

Chat models offer a standard set of parameters that can be used to configure the model. These parameters are typically u

## Using Retrievers in LCEL

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOllama(model='llama3.2:1b')


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke("What did the president say about technology?"))


Based on the context, it appears that the question is related to a text document stored in a vector store. The vector store embeds this text content and allows for similarity searches based on keyword similarities.

To answer your question, we need to search for documents (specifically those with metadata "source" = "tweet") within the vector store that contain keywords related to technology, such as "technology", "innovation", etc.

Here's how you can do it:

```python
vectorstore.similarity_search(
    query="What did the president say about technology?",
    k=2,
    filter={"metadata": {"source": "tweet"}},
)
```

This will return a list of documents that contain keywords related to technology in at least two similar ways to the search query.
